# Compare SQLs

# Dataset

In [1]:
import pandas as pd
# load csv
db_train = pd.read_csv('llm/data/txt2sql_alerce_train_v4_0.csv')
db_test = pd.read_csv('llm/data/txt2sql_alerce_test_v4_0.csv')

In [2]:
import os
import sqlalchemy as sa
import requests
import time
# Create a connection to the ALeRCE database
def create_conn_alerce(access_time: int = 2):
  """
  Create a connection to the ALeRCE database.
  
  Args:
    access_time: Integer representing the time limit for the connection.
    2 for default access, 10 for extended access.
    
  Returns:
    SQLAlchemy engine object
  
  Raises:
    ValueError: If URL fetch fails or credentials are invalid
  """
  # Common URL for both access levels
  url = "https://raw.githubusercontent.com/alercebroker/usecases/master/alercereaduser_v4.json"
  
  # For security, get extended access credentials from environment variables
  user_extended = os.environ.get('ALERCE_USER_EXTENDED', '')
  pass_extended = os.environ.get('ALERCE_PASS_EXTENDED', '')
  
  n_tries = 3
  params = None
  
  # Fetch parameters from URL with retry logic
  for n_try in range(n_tries):
    try:
      response = requests.get(url)
      if response.status_code != 200:
        if n_try < n_tries - 1:
          time.sleep(2 ** n_try)  # exponential backoff
          continue
        else:
          raise ValueError(f"Failed to fetch URL: {url}, Status Code: {response.status_code}")
      
      params = response.json().get('params')
      if not params:
        raise ValueError("Missing 'params' in the JSON response")
      break
        
    except requests.RequestException as e:
      if n_try < n_tries - 1:
        time.sleep(2 ** n_try)
        continue
      else:
        raise ValueError(f"Network error when fetching {url}: {str(e)}")
    except ValueError as e:
      if "JSON" in str(e):
        raise ValueError("Invalid JSON response from URL")
      else:
        raise e
  
  # Create connection string based on access level
  if access_time == 2:
    conn_string = f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}"
  elif access_time == 10:
    if not user_extended or not pass_extended:
      raise ValueError("Extended access credentials not found in environment variables")
    conn_string = f"postgresql+psycopg2://{user_extended}:{pass_extended}@{params['host']}/{params['dbname']}"
  else:
    raise ValueError(f'Access time {access_time} not supported')
  
  # Create and return engine with connection pooling disabled
  engine = sa.create_engine(conn_string, poolclass=sa.pool.NullPool)
  return engine


def run_sql_alerce(
    sql: str, 
    access_time: int = 2, 
    n_tries: int = 3, 
    query_time: bool = False
):
  ''' Execute the SQL query at the ALeRCE database and return the result
    Args:
        sql: SQL query to execute
        access_time: Integer representing the time limit for the connection. 
        2 for default access, 10 for extended access.
        n_tries: Number of tries to execute the query
        query_time: Boolean indicating whether to track query execution time
    Returns:
    query: pandas dataframe with the result of the query
    error: error message if the query could not be executed
    execution_time: time taken to execute the query    
  '''

  try:
    engine = create_conn_alerce(access_time=access_time)
  except ValueError as e:
    return None, e
    
  query = None
  error = None
  execution_time = None
  
  try:
    for n_try in range(n_tries):
      try:
        with engine.begin() as conn:
          start_time = time.time()
          query = pd.read_sql_query(sa.text(sql), conn)
          if query_time:
            execution_time = time.time() - start_time
          error = None
          break
      except Exception as e:
        error = e
        if n_try < n_tries - 1:
          time.sleep(2 ** n_try)  # exponential backoff
        else:
          # Last attempt failed, keep the error
          pass
  
  finally:
    # Always dispose of the engine to close connections
    engine.dispose()

  if query_time:
    return query, error, execution_time
  else:
    return query, error




## Running test of train and test queries


In [3]:
gold_columns_train = []
# iter db_train 
n = 5
for indx, row in db_train.iterrows():
    
    query = row['gold_query']
    tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
    if error:
        print(f'Error in query {row.req_id}: {error}')
        gold_columns_train.append({"req_id": row.req_id, "cols": [], "shape": []})
    else:
        print(f'Query {row.req_id} executed successfully, shape: {tables_i.shape}')
        gold_columns_train.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})

Query 13 executed successfully, shape: (4471, 4)
Query 10 executed successfully, shape: (184, 3)
Query 15 executed successfully, shape: (42, 7)
Query 4 executed successfully, shape: (1537, 8)
Query 25 executed successfully, shape: (4, 30)
Query 7 executed successfully, shape: (896, 2)
Query 2 executed successfully, shape: (5732, 2)
Query 3 executed successfully, shape: (11, 6)
Query 23 executed successfully, shape: (668355, 28)
Query 32 executed successfully, shape: (2, 5)
Query 20 executed successfully, shape: (41262, 28)
Query 18 executed successfully, shape: (1041, 11)
Query 35 executed successfully, shape: (10, 6)
Query 11 executed successfully, shape: (491, 4)
Query 9 executed successfully, shape: (54, 8)
Query 5 executed successfully, shape: (781, 9)
Query 0 executed successfully, shape: (8218, 6)
Query 28 executed successfully, shape: (34, 1)
Query 36 executed successfully, shape: (178, 3)
Query 38 executed successfully, shape: (1247, 22)
Query 24 executed successfully, shape: (

In [4]:
gold_columns_test = []
# iter db_test 
n = 5
for indx, row in db_test.iterrows():
    
    query = row['gold_query']
    tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
    if error:
        print(f'Error in query {row.req_id}: {error}')
        gold_columns_test.append({"req_id": row.req_id, "cols": [], "shape": []})
    else:
        print(f'Query {row.req_id} executed successfully, shape: {tables_i.shape}')
        gold_columns_test.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})

Query 8 executed successfully, shape: (186, 28)
Query 27 executed successfully, shape: (5520, 11)
Error in query 17: (psycopg2.errors.QueryCanceled) canceling statement due to statement timeout

[SQL: 
SELECT
  *
FROM
  (
SELECT *
FROM (
SELECT
  feature.oid, feature.value, feature.name, feature.version, feature.fid
FROM
  (
SELECT
    object.oid, object.meanra, object.meandec
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='lc_classifier'
    AND probability.class_name = 'AGN'
    AND probability.ranking = 1
) as obj_oids
    INNER JOIN
    feature ON feature.oid = obj_oids.oid
WHERE
  feature.name = 'ExcessVar'
  AND feature.fid = 1
  AND feature.value < -0.001
ORDER BY feature.value ASC
) as sq1
UNION
SELECT *
FROM (
SELECT
  feature.oid, feature.value, feature.name, feature.version, feature.fid
FROM
  (
SELECT
    object.oid, object.meanra, object.meandec
FROM
    object INNER JOIN
    probability
    ON object.oi

## Columns

In [5]:
gold_columns_train

[{'req_id': 13,
  'cols': ['oid', 'class_name', 'lastmjd', 'firstmjd'],
  'shape': (4471, 4)},
 {'req_id': 10, 'cols': ['oid', 'sn_prob', 'cv_prob'], 'shape': (184, 3)},
 {'req_id': 15,
  'cols': ['oid',
   'probability',
   'candid',
   'fid',
   'mjd',
   'magstat_fid',
   'dmdt_first'],
  'shape': (42, 7)},
 {'req_id': 4,
  'cols': ['oid',
   'fid',
   'dmdt_first',
   'candid',
   'f_id',
   'magpsf',
   'sigmapsf_corr',
   'sigmapsf_corr_ext'],
  'shape': (1537, 8)},
 {'req_id': 25,
  'cols': ['oid',
   'ndethist',
   'ncovhist',
   'mjdstarthist',
   'mjdendhist',
   'corrected',
   'stellar',
   'ndet',
   'g_r_max',
   'g_r_max_corr',
   'g_r_mean',
   'g_r_mean_corr',
   'meanra',
   'meandec',
   'sigmara',
   'sigmadec',
   'deltajd',
   'firstmjd',
   'lastmjd',
   'step_id_corr',
   'diffpos',
   'reference_change',
   'oid',
   'classifier_name',
   'class_name',
   'probability',
   'ranking',
   'source_id',
   'ra',
   'dec'],
  'shape': (4, 30)},
 {'req_id': 7, 'cols'

In [6]:
gold_columns_test


[{'req_id': 8,
  'cols': ['oid',
   'ndethist',
   'ncovhist',
   'mjdstarthist',
   'mjdendhist',
   'corrected',
   'stellar',
   'ndet',
   'g_r_max',
   'g_r_max_corr',
   'g_r_mean',
   'g_r_mean_corr',
   'meanra',
   'meandec',
   'sigmara',
   'sigmadec',
   'deltajd',
   'firstmjd',
   'lastmjd',
   'step_id_corr',
   'diffpos',
   'reference_change',
   'oid',
   'classifier_name',
   'classifier_version',
   'class_name',
   'probability',
   'ranking'],
  'shape': (186, 28)},
 {'req_id': 27,
  'cols': ['oid',
   'meanra',
   'meandec',
   'ndet',
   'firstmjd',
   'deltajd',
   'g_r_max',
   'classifier_name',
   'class_name',
   'ranking',
   'probability'],
  'shape': (5520, 11)},
 {'req_id': 17, 'cols': [], 'shape': []},
 {'req_id': 37,
  'cols': ['oid',
   'classifier_name',
   'classifier_version',
   'class_name',
   'probability',
   'ranking'],
  'shape': (200, 6)},
 {'req_id': 34,
  'cols': ['oid', 'name', 'value', 'fid', 'version'],
  'shape': (354, 5)},
 {'req_id

## Test single query

In [21]:

query = '''
SELECT
    det.oid, det.candid,
    det.drb, det.fid,
    det.mjd, det.magpsf, det.sigmapsf,
    det.has_stamp,
    ps1.sgscore1, ps1.distpsnr1
FROM
    (SELECT *
    FROM detection
    WHERE oid in ('ZTF23aavzgjg','ZTF23aaynzyk','ZTF23aavqxos','ZTF23aaknyni','ZTF23aavsdtc','ZTF18aandkua','ZTF23aaxfewt','ZTF23aavshwi','ZTF22aawasao','ZTF23aaxgvnt')
    ) as det
    INNER JOIN
    (SELECT *
    FROM ps1_ztf
    WHERE oid in ('ZTF23aavzgjg','ZTF23aaynzyk','ZTF23aavqxos','ZTF23aaknyni','ZTF23aavsdtc','ZTF18aandkua','ZTF23aaxfewt','ZTF23aavshwi','ZTF22aawasao','ZTF23aaxgvnt')
    ) as ps1
    ON det.oid=ps1.oid
WHERE
     (ps1.sgscore1 < 0.5 OR ps1.distpsnr1 < 1)
    AND det.drb > 0.5

'''
query_columns = []
n = 3
tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
if error:
    print(f'Error in query: {error}')
    # query_columns.append({"req_id": row.req_id, "cols": [], "shape": []})
else:
    print(f'Query executed successfully, shape: {tables_i.shape}')
    # query_columns.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})
tables_i

Query executed successfully, shape: (30, 10)


,oid,candid,drb,fid,mjd,magpsf,sigmapsf,has_stamp,sgscore1,distpsnr1
0,ZTF18aandkua,2434183903015015056,0.994159,1,60188.183901,17.907194,0.180156,True,0.075167,0.326697
1,ZTF18aandkua,2756269351915010002,0.993011,2,60510.269352,19.379133,0.161114,True,0.075167,0.326697
2,ZTF18aandkua,2772265531915010009,0.850113,2,60526.265532,19.641638,0.183068,True,0.075167,0.326697
3,ZTF18aandkua,2979404491915010015,0.696538,1,60733.404491,18.887863,0.175351,True,0.075167,0.326697
4,ZTF18aandkua,2434183903015015056,0.994159,1,60188.183901,17.907194,0.180156,True,0.075167,1.234589
5,ZTF18aandkua,2756269351915010002,0.993011,2,60510.269352,19.379133,0.161114,True,0.075167,1.234589
6,ZTF18aandkua,2772265531915010009,0.850113,2,60526.265532,19.641638,0.183068,True,0.075167,1.234589
7,ZTF18aandkua,2979404491915010015,0.696538,1,60733.404491,18.887863,0.175351,True,0.075167,1.234589
8,ZTF22aawasao,2754319833415015025,0.755912,2,60508.319838,18.594233,0.143780,True,0.380149,0.673811
9,ZTF22aawasao,2754319833415015025,0.755912,2,60508.319838,18.594233,0.143780,True,0.380149,0.904783


In [20]:
query =  '''
-- Query to retrieve the required information for the specified oids
WITH selected_oids AS (-- Define the list of oids as a temporary table

                       VALUES ('ZTF23aavzgjg'), ('ZTF23aaynzyk'), ('ZTF23aavqxos'), ('ZTF23aaknyni'), ('ZTF23aavsdtc'), ('ZTF18aandkua'), ('ZTF23aaxfewt'), ('ZTF23aavshwi'), ('ZTF22aawasao'), ('ZTF23aaxgvnt'))
SELECT d.oid, -- Unique object identifier
 d.candid, -- Candidate identifier
 d.fid, -- Filter identifier
 d.mjd, -- Modified Julian Date
 d.magpsf AS magnitude, -- Magnitude
 d.sigmapsf AS magnitude_error, -- Magnitude error
 d.has_stamp, -- Whether the object has stamps
 d.drb, -- Deep learning real bogus score
 p.sgscore1, -- Star galaxy score of the nearest object
 p.distpsnr1 -- Distance to the nearest source in Pan-STARRS
FROM detection d
JOIN ps1_ztf p ON d.oid = p.oid -- Join detection and ps1_ztf tables on oid

WHERE d.oid IN
    (SELECT column1
     FROM selected_oids)-- Filter for the specified oids

  AND d.drb > 0.5 -- Deep learning real bogus score greater than 0.5

  AND (p.sgscore1 < 0.5
       OR p.distpsnr1 < 1); -- Either star galaxy score < 0.5 or distance to nearest source < 1 arcsec
'''
n = 3
tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
if error:
    print(f'Error in query: {error}')
    # query_columns.append({"req_id": row.req_id, "cols": [], "shape": []})
else:
    print(f'Query executed successfully, shape: {tables_i.shape}')
    # query_columns.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})
tables_i

Query executed successfully, shape: (30, 10)


,oid,candid,fid,mjd,magnitude,magnitude_error,has_stamp,drb,sgscore1,distpsnr1
0,ZTF23aavshwi,2448496784015015039,2,60202.496782,20.105957,0.146235,True,0.999999,0.009583,2.024725
1,ZTF23aavshwi,2446471444015015013,2,60200.471447,20.102201,0.131913,True,0.999977,0.009583,2.024725
2,ZTF23aavshwi,2440490024015015015,2,60194.490023,19.956028,0.172092,True,0.999963,0.009583,2.024725
3,ZTF23aavshwi,2432427924015015008,2,60186.427928,19.566288,0.176278,True,0.999994,0.009583,2.024725
4,ZTF23aavshwi,2430476444015015012,2,60184.476447,19.384290,0.112882,True,0.999998,0.009583,2.024725
5,ZTF23aavshwi,2430436074015015016,1,60184.436076,20.503382,0.193578,True,0.999985,0.009583,2.024725
6,ZTF23aavshwi,2428469354015015016,2,60182.469352,19.151363,0.086399,True,1.000000,0.009583,2.024725
7,ZTF23aavshwi,2424486604015015016,2,60178.486609,18.973866,0.079126,True,1.000000,0.009583,2.024725
8,ZTF23aavshwi,2419475694015015014,1,60173.475694,19.894947,0.147604,True,0.999999,0.009583,2.024725
9,ZTF23aavshwi,2419451634015015008,2,60173.451632,18.789606,0.067233,True,1.000000,0.009583,2.024725


In [19]:
query =  '''
-- Query to retrieve the required information for the specified oids
SELECT 
    d.oid, -- Unique object identifier
    d.candid, -- Candidate identifier
    d.fid AS filter_id, -- Filter identifier
    d.mjd AS modified_julian_date, -- Modified Julian Date
    d.magpsf AS magnitude, -- Magnitude
    d.sigmapsf AS magnitude_error, -- Magnitude error
    d.has_stamp, -- Whether the object has stamps
    d.drb AS deep_learning_real_bogus_score, -- Deep learning real bogus score
    pz.sgscore1 AS star_galaxy_score_nearest, -- Star galaxy score of the nearest object
    pz.distpsnr1 AS distance_to_nearest_source -- Distance to the nearest source in Pan-STARRS
FROM 
    detection d
WHERE 
    d.oid IN (
        'ZTF23aavzgjg', 'ZTF23aaynzyk', 'ZTF23aavqxos', 'ZTF23aaknyni',
        'ZTF23aavsdtc', 'ZTF18aandkua', 'ZTF23aaxfewt', 'ZTF23aavshwi',
        'ZTF22aawasao', 'ZTF23aaxgvnt'
    ) -- Filter by the specified list of oids
    AND d.drb > 0.5 -- Deep learning real bogus score greater than 0.5
    AND (
        pz.sgscore1 < 0.5 -- Star galaxy score less than 0.5
        OR pz.distpsnr1 < 1 -- Distance to the nearest Pan-STARRS source smaller than 1 arcsec
    )
    AND EXISTS (
        SELECT 1
        FROM ps1_ztf pz
        WHERE pz.oid = d.oid
    ) -- Ensure the object exists in the ps1_ztf table
'''
n = 3
tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
if error:
    print(f'Error in query: {error}')
    # query_columns.append({"req_id": row.req_id, "cols": [], "shape": []})
else:
    print(f'Query executed successfully, shape: {tables_i.shape}')
    # query_columns.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})
tables_i

Error in query: (psycopg2.errors.UndefinedTable) missing FROM-clause entry for table "pz"
LINE 12:     pz.sgscore1 AS star_galaxy_score_nearest, -- Star galaxy...
             ^

[SQL: 
-- Query to retrieve the required information for the specified oids
SELECT 
    d.oid, -- Unique object identifier
    d.candid, -- Candidate identifier
    d.fid AS filter_id, -- Filter identifier
    d.mjd AS modified_julian_date, -- Modified Julian Date
    d.magpsf AS magnitude, -- Magnitude
    d.sigmapsf AS magnitude_error, -- Magnitude error
    d.has_stamp, -- Whether the object has stamps
    d.drb AS deep_learning_real_bogus_score, -- Deep learning real bogus score
    pz.sgscore1 AS star_galaxy_score_nearest, -- Star galaxy score of the nearest object
    pz.distpsnr1 AS distance_to_nearest_source -- Distance to the nearest source in Pan-STARRS
FROM 
    detection d
WHERE 
    d.oid IN (
        'ZTF23aavzgjg', 'ZTF23aaynzyk', 'ZTF23aavqxos', 'ZTF23aaknyni',
        'ZTF23aavsdtc', 'ZTF18aan

In [ ]:
# query 96
query = '''-- Final Query to retrieve all detections for ZTF objects within 2 arcseconds of Solar System identifiers '2003FP134' and '2009UK56'
WITH solar_system_objects AS ( -- Define the RA/Dec coordinates for the Solar System identifiers 
SELECT DISTINCT ssnamenr, oid, candid, ssdistnr, ssmagnr FROM ss_ztf WHERE ssnamenr IN ('2003FP134', '2009UK56')),
filtered_objects AS ( -- Perform spatial filtering using q3c_radial_query to find objects within 2 arcseconds 
SELECT sso.*, d.* FROM solar_system_objects sso INNER JOIN detection d ON sso.candid = d.candid 
WHERE q3c_radial_query(d.ra, d.dec, sso.ssdistnr, 0.0005556) -- 2 arcseconds in degrees
) -- Select required columns and sort the results
SELECT  sso.*,  d.mjd AS detection_date,  d.fid AS filter_identifier,  d.isdiffpos,  d.ra,  d.dec,  d.magpsf AS difference_magnitude,  d.sigmapsf AS magnitude_uncertainty 
FROM filtered_objects sso
ORDER BY sso.ssnamenr, d.mjd'''

query = '''-- Final Query to retrieve all detections for ZTF objects within 2 arcseconds of Solar System identifiers '2003FP134' and '2009UK56'
WITH catalog AS (    -- Define the Solar System objects of interest with their RA/Dec coordinates    
SELECT '2003FP134' AS source_id, ra_value1 AS ra, dec_value1 AS dec
UNION ALL    SELECT '2009UK56' AS source_id, ra_value2 AS ra, dec_value2 AS dec)
SELECT     ss_ztf.*, -- All columns from the ss_ztf table    
detection.mjd AS detection_date, -- Detection date
detection.fid AS filter_identifier, -- Filter identifier
detection.isdiffpos, -- Isdiffpos flag
detection.ra, -- RA coordinate
detection.dec, -- Dec coordinate
detection.magpsf AS difference_magnitude, -- Difference magnitude    
detection.sigmapsf AS magnitude_uncertainty -- Magnitude uncertainty
FROM     ss_ztf
INNER JOIN     catalog    ON q3c_radial_query(ss_ztf.ra, ss_ztf.dec, catalog.ra, catalog.dec, 0.0005556) -- Spatial filtering within 2 arcseconds
INNER JOIN     detection    ON ss_ztf.oid = detection.oid -- Join with detection table on oid
WHERE     ss_ztf.ssnamenr IN ('2003FP134', '2009UK56') -- Filter for specific Solar System objects
ORDER BY     ss_ztf.ssnamenr ASC, -- Sort by Solar System object name    
detection.mjd ASC -- Sort by detection date'''

query = '''-- [FINAL QUERY HERE]
SELECT ss_ztf.*, detection.mjd AS detection_date, detection.fid AS filter_identifier, detection.isdiffpos, detection.ra, detection.dec, 
detection.magpsf AS difference_magnitude, detection.sigmapsf AS magnitude_uncertainty
FROM ss_ztf
INNER JOIN detection ON ss_ztf.oid = detection.oid
WHERE ss_ztf.ssnamenr IN ('2003FP134', '2009UK56')
AND q3c_radial_query(detection.ra, detection.dec, ss_ztf.ra, ss_ztf.dec, 0.0005556)
ORDER BY ss_ztf.ssnamenr ASC, detection.mjd ASC'''


n = 3
tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
if error:
    print(f'Error in query: {error}')
    # query_columns.append({"req_id": row.req_id, "cols": [], "shape": []})
else:
    print(f'Query executed successfully, shape: {tables_i.shape}')
    # query_columns.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})
tables_i

Query executed successfully, shape: (1, 7)


,oid,candid,neargaia,neargaiabright,maggaia,maggaiabright,unique1
0,ZTF18acxlskz,1917201552615010028,0.74125,-999.0,15.321053,-999.0,False


In [8]:
db_test[db_test.req_id == 90].request.values[0]

"Find at most 100 ZTF objects that have a multiband period lower than 5 days in the 'lc_classifier_1.2.1-P' feature version. Return all columns from the 'probability' table for such objects, including only data for the light curve classifier, with rankings either 1 or 2"

In [12]:
print(db_test[db_test.req_id == 90].gold_query.values[0])

SELECT
    *
FROM
    probability
WHERE
    classifier_name = 'lc_classifier'
    AND ranking <= 2
    AND oid in
(
SELECT
    oid
FROM
    feature
WHERE
    name = 'Multiband_period'
    AND version = 'lc_classifier_1.2.1-P'
    AND value < 5
LIMIT 100
)


In [36]:
db_test[db_test.req_id == 90].request.values[0]


"Find at most 100 ZTF objects that have a multiband period lower than 5 days in the 'lc_classifier_1.2.1-P' feature version. Return all columns from the 'probability' table for such objects, including only data for the light curve classifier, with rankings either 1 or 2"

In [30]:
# query 90
query = '''SELECT DISTINCT ON (oid)
    *
FROM
    probability
WHERE
    classifier_name = 'lc_classifier'
    AND ranking <= 2
    AND oid in
(
SELECT
    oid
FROM
    feature
WHERE
    name = 'Multiband_period'
    AND version = 'lc_classifier_1.2.1-P'
    AND value < 5
LIMIT 100
)'''
# # run 1 simon 
# # mutiband_period with lower case
# # limit 100 at the end, not inside the subquery
# query='''
# -- Final Query to retrieve at most 100 ZTF objects with a multiband period lower than 5 days
# -- in the 'lc_classifier_1.2.1-P' feature version, returning all columns from the 'probability' table
# -- for the light curve classifier with rankings 1 or 2.
# WITH filtered_features AS (    -- Subquery 1: Filter objects based on multiband period
#     SELECT oid    FROM feature    WHERE name = 'multiband_period'      AND value < 5
#     AND version = 'lc_classifier_1.2.1-P'),
#     filtered_probabilities AS (    -- Subquery 2: Filter probabilities for the light curve classifier
#     SELECT *    FROM probability    WHERE classifier_name = 'lc_classifier'
#     AND ranking IN (1, 2)) -- Combine the filters using an INNER JOIN
#     SELECT p.*
#     FROM filtered_probabilities p
#     INNER JOIN filtered_features f
#     ON p.oid = f.oid-- Limit the results to at most 100 objects
#     LIMIT 100
# '''
# # run 2 simon
# # mutiband_period with upper case
# # limit 100 at the end, not inside the subquery
# query='''
# -- Final Query to retrieve at most 100 ZTF objects with a multiband period lower than 5 days
# -- in the 'lc_classifier_1.2.1-P' feature version, returning all columns from the 'probability' table
# -- for the light curve classifier with rankings 1 or 2.
# WITH filtered_features AS (    -- Subquery 1: Filter objects based on multiband period
#     SELECT oid    FROM feature
#         WHERE name = 'Multiband_period'
#               AND value < 5
#               AND version = 'lc_classifier_1.2.1-P'),
#               filtered_probabilities AS (    -- Subquery 2: Filter probabilities for the light curve classifier
#                 SELECT *    FROM probability    
#                 WHERE classifier_name = 'lc_classifier'      
#                 AND ranking IN (1, 2))-- Combine the filters using an INNER JOIN
#                 SELECT p.*
#                 FROM filtered_probabilities p
#                 INNER JOIN filtered_features f
#                 ON p.oid = f.oid -- Limit the results to at most 100 objects
#                 LIMIT 100
# '''
# # run 3 simon
# query='''
# -- Final Query to retrieve at most 100 ZTF objects with a multiband period lower than 5 days
# -- in the 'lc_classifier_1.2.1-P' feature version, returning all columns from the 'probability' table
# -- for the light curve classifier with rankings of 1 or 2.
# WITH filtered_features AS (    -- Subquery 1: Filter the 'feature' table for objects with multiband_period < 5
#     SELECT oid    FROM feature    
#     WHERE name = 'Multiband_period'      
#     AND value < 5      
#     AND version = 'lc_classifier_1.2.1-P'),
#     filtered_probabilities AS (    -- Subquery 2: Filter the 'probability' table for the light curve classifier with rankings 1 or 2
#         SELECT *    FROM probability    
#         WHERE classifier_name = 'lc_classifier'      
#         AND (ranking = 1 OR ranking = 2))-- Final Query: Join the filtered results and limit to 100 rows
#         SELECT p.*
#         FROM filtered_probabilities p
#         INNER JOIN filtered_features f
#         ON p.oid = f.oid
#         LIMIT 100
# '''
n = 3
tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
if error:
    print(f'Error in query: {error}')
    # query_columns.append({"req_id": row.req_id, "cols": [], "shape": []})
else:
    print(f'Query executed successfully, shape: {tables_i.shape}')
    # query_columns.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})
tables_i

Query executed successfully, shape: (100, 6)


,oid,classifier_name,classifier_version,class_name,probability,ranking
0,ZTF17aaaessu,lc_classifier,lc_classifier_1.1.13,CEP,0.225984,2
1,ZTF17aaagqxl,lc_classifier,hierarchical_rf_1.1.0,E,0.317112,1
2,ZTF17aabwftj,lc_classifier,lc_classifier_1.1.13,Periodic-Other,0.262268,1
3,ZTF17aacpvbf,lc_classifier,hierarchical_rf_1.1.0,E,0.494016,1
4,ZTF18aaydigk,lc_classifier,hierarchical_rf_1.1.0,YSO,0.494768,1
...,...,...,...,...,...,...
95,ZTF21aaaqswa,lc_classifier,hierarchical_rf_1.1.0,AGN,0.132756,2
96,ZTF21aaguuxw,lc_classifier,hierarchical_rf_1.1.0,Periodic-Other,0.280500,1
97,ZTF21abcdsuv,lc_classifier,hierarchical_rf_1.1.0,E,0.426688,1
98,ZTF22aabwnvh,lc_classifier,hierarchical_rf_1.1.0,QSO,0.678160,1


In [19]:
# query 90
query = '''SELECT 
    *
FROM
    probability
WHERE
    classifier_name = 'lc_classifier'
    AND ranking <= 2
    AND oid in
(
SELECT
    oid
FROM
    feature
WHERE
    name = 'Multiband_period'
    AND version = 'lc_classifier_1.2.1-P'
    AND value < 5
LIMIT 100
)'''
n = 3
tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
if error:
    print(f'Error in query: {error}')
    # query_columns.append({"req_id": row.req_id, "cols": [], "shape": []})
else:
    print(f'Query executed successfully, shape: {tables_i.shape}')
    # query_columns.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})
tables_i

Query executed successfully, shape: (346, 6)


,oid,classifier_name,classifier_version,class_name,probability,ranking
0,ZTF19abdyprc,lc_classifier,lc_classifier_1.1.13,CEP,0.161280,2
1,ZTF19abdyprc,lc_classifier,hierarchical_rf_1.1.0,DSCT,0.283360,1
2,ZTF19abdyprc,lc_classifier,lc_classifier_1.1.13,RRL,0.190720,1
3,ZTF19abdyprc,lc_classifier,hierarchical_rf_1.1.0,RRL,0.227040,2
4,ZTF18abadjvy,lc_classifier,hierarchical_rf_1.1.0,E,0.093744,2
...,...,...,...,...,...,...
341,ZTF18abasovn,lc_classifier,hierarchical_rf_1.1.0,E,0.424664,1
342,ZTF18abasovn,lc_classifier,lc_classifier_1.1.13,E,0.309624,1
343,ZTF18abasovn,lc_classifier,hierarchical_rf_1.1.0,RRL,0.188956,2
344,ZTF22aabwnvh,lc_classifier,hierarchical_rf_1.1.0,Blazar,0.205800,2


In [29]:
sorted(tables_i.oid.unique())

['ZTF17aaaessu',
 'ZTF17aaagqxl',
 'ZTF17aabwftj',
 'ZTF17aacpvbf',
 'ZTF18aaydigk',
 'ZTF18aayhppd',
 'ZTF18aaynqnn',
 'ZTF18aaynqox',
 'ZTF18aaypmtr',
 'ZTF18aayujpj',
 'ZTF18aayutyl',
 'ZTF18aayuugb',
 'ZTF18aayvacc',
 'ZTF18aayvbrz',
 'ZTF18aazwigp',
 'ZTF18abadjvy',
 'ZTF18abaqmri',
 'ZTF18abasovn',
 'ZTF18abcxguj',
 'ZTF18abgsptz',
 'ZTF18abilxvt',
 'ZTF18abivpiy',
 'ZTF18abkjhnt',
 'ZTF18abkmmts',
 'ZTF18ablpmbs',
 'ZTF18ablqndm',
 'ZTF18ablvyyw',
 'ZTF18abmjosl',
 'ZTF18abmrcsn',
 'ZTF18abommtg',
 'ZTF18aboskew',
 'ZTF18abpogdx',
 'ZTF18abrnfpd',
 'ZTF18abrrinj',
 'ZTF18abscvmd',
 'ZTF18absitkp',
 'ZTF18absoocn',
 'ZTF18abspqgp',
 'ZTF18abvgrec',
 'ZTF18abvztqd',
 'ZTF18acajzhc',
 'ZTF18acavigy',
 'ZTF18acdasee',
 'ZTF18acfrjme',
 'ZTF18achfgss',
 'ZTF18achixag',
 'ZTF18aciepmk',
 'ZTF18acnambs',
 'ZTF18acrugci',
 'ZTF18acsbrqc',
 'ZTF18actytgu',
 'ZTF18acuwvpr',
 'ZTF18acwbhzi',
 'ZTF18adamiis',
 'ZTF18adamzmn',
 'ZTF18adaoqcm',
 'ZTF18adapbif',
 'ZTF18adapnnv',
 'ZTF18adaqdfq

In [31]:
sorted(tables_i.oid.unique())

['ZTF17aaaessu',
 'ZTF17aaagqxl',
 'ZTF17aabwftj',
 'ZTF17aacpvbf',
 'ZTF18aaydigk',
 'ZTF18aayhppd',
 'ZTF18aaynqnn',
 'ZTF18aaynqox',
 'ZTF18aaypmtr',
 'ZTF18aayujpj',
 'ZTF18aayutyl',
 'ZTF18aayuugb',
 'ZTF18aayvacc',
 'ZTF18aayvbrz',
 'ZTF18aazwigp',
 'ZTF18abadjvy',
 'ZTF18abaqmri',
 'ZTF18abasovn',
 'ZTF18abcxguj',
 'ZTF18abgsptz',
 'ZTF18abilxvt',
 'ZTF18abivpiy',
 'ZTF18abkjhnt',
 'ZTF18abkmmts',
 'ZTF18ablpmbs',
 'ZTF18ablqndm',
 'ZTF18ablvyyw',
 'ZTF18abmjosl',
 'ZTF18abmrcsn',
 'ZTF18abommtg',
 'ZTF18aboskew',
 'ZTF18abpogdx',
 'ZTF18abrnfpd',
 'ZTF18abrrinj',
 'ZTF18abscvmd',
 'ZTF18absitkp',
 'ZTF18absoocn',
 'ZTF18abspqgp',
 'ZTF18abvgrec',
 'ZTF18abvztqd',
 'ZTF18acajzhc',
 'ZTF18acavigy',
 'ZTF18acdasee',
 'ZTF18acfrjme',
 'ZTF18achfgss',
 'ZTF18achixag',
 'ZTF18aciepmk',
 'ZTF18acnambs',
 'ZTF18acrugci',
 'ZTF18acsbrqc',
 'ZTF18actytgu',
 'ZTF18acuwvpr',
 'ZTF18acwbhzi',
 'ZTF18adamiis',
 'ZTF18adamzmn',
 'ZTF18adaoqcm',
 'ZTF18adapbif',
 'ZTF18adapnnv',
 'ZTF18adaqdfq

In [39]:
# query 84
query = '''
SELECT
    oid, xmatch.oid_catalog, dist,
    w1mpro, w2mpro, w3mpro, w4mpro
FROM
    xmatch INNER JOIN
    allwise
    ON xmatch.oid_catalog = allwise.oid_catalog
WHERE
    xmatch.oid = 'ZTF19aascdol'
'''
# # run 2 simon 
query = '''
-- Retrieve information about the ZTF object 'ZTF19aascdol' and its ALLWISE match(es)
SELECT     object.oid AS ztf_identifier, -- ZTF object identifier    
allwise.oid_catalog AS allwise_identifier, -- ALLWISE catalog identifier    
q3c_dist(object.meanra, object.meandec, allwise.ra, allwise.dec) AS distance, -- Angular distance between counterparts    
allwise.w1mpro, -- WISE W1 magnitude    
allwise.w2mpro, -- WISE W2 magnitude    
allwise.w3mpro, -- WISE W3 magnitude    
allwise.w4mpro  -- WISE W4 magnitude
FROM     object
INNER JOIN     allwise
ON     q3c_join(object.meanra, object.meandec, allwise.ra, allwise.dec, 0.00138889) -- Match within 5 arcseconds (0.00138889 degrees)
WHERE     object.oid = 'ZTF19aascdol' -- Filter for the specific ZTF object
ORDER BY     distance ASC -- Sort by angular distance to prioritize the closest match
'''
n = 3
tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
if error:
    print(f'Error in query: {error}')
    # query_columns.append({"req_id": row.req_id, "cols": [], "shape": []})
else:
    print(f'Query executed successfully, shape: {tables_i.shape}')
    # query_columns.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})
tables_i

Query executed successfully, shape: (1, 7)


,ztf_identifier,allwise_identifier,distance,w1mpro,w2mpro,w3mpro,w4mpro
0,ZTF19aascdol,J154334.93+152539.5,0.000038,15.286,14.354,11.747,8.611
